# RNA-Seq Hands-on

## Introduction

### Intended learning outcomes

This tutorial is meant to show you the basics of a bulk tissue RNA-Seq differential expression analysis. Today, you will learn...

* terminology of bulk RNA-Seq bioinformatics analyses.
* to name some important bulk RNA-Seq tools and describe their basic functionalities.
* to name and explain the steps of a bulk RNA-Seq analysis workflow.
* to explain the most important plot types and how to plot and visualize bulk RNA-Seq results with ggplot2.
* to apply R code to your bulk RNA-Seq data to identify differentially expressed genes, plot results, and find enriched gene ontology terms.
* to interpret the results you get from bulk RNA-Seq analyses.

The nuances of bulk RNA-Seq data generation like experimental design and choice of library preparation methods are not part of this lecture and tutorial. If you'd like to look into that, we designed an OLAT-course which takes approx. 60 minutes on that topic. You can find it at https://lms.uni-kiel.de/url/Catalog/0/Search/0/Infos/5273846135 using biomedinf2022 as a password.

### Roadmap

This is the roadmap for today.

1. Initialize R environment
2. Inspect FastQC results
3. Read files, data preprocessing
4. Exploratory data analysis with PCA
5. 
    a. Differential expression analysis, using DESeq2
    b. Volcano plot
    c. Heatmap
6. Enrichment analysis of differentially expressed genes

Most of the sections contain tasks. The tasks under the "Tasks" headings are basic tasks to test yourselves if you understood what just happened in the code. The "Expert Questions" tasks are a little more complicated and require some fiddling with code. We recommend to go through the script and the basic tasks first, and then continuing with advanced tasks you find interesting.

Try to solve the tasks without directly looking up the solution to improve your learning. Try it below!

#### Tasks

Example task: Which tissue was sampled to generate the data in this tutorial?

The data was generated from full peripheral blood samples.

### Terminology

Let's briefly review the terminology:

* *bulk tissue* as opposed to *single-cell* means that a tissue sample consisting of many cells and possibly different cell types was analyzed. The tissue used here is full peripheral blood.
* *differential expression* means that the relative abundance of many types of transcripts is compared between samples. Here we compare samples of healthy donors and patients infected with COVID-19 in different stages of the disease.

### Workflow

The bulk RNA-Seq workflow can be thought as a procedure comprising these steps:

1. Library prep and Sequencing
2. Quality control of sequencing reads
3. Gapped alignment of sequencing reads to a reference genome
4. Feature counting (counting reads with genes as bins)
5. Differential expression analysis
6. Gene ontology enrichment analysis

This hands-on tutorial focuses on steps 5. and 6. We did the steps 1.-4. for you using the nf-core/rnaseq pipeline. These steps can be standardized and pipelining them saves us a lot of time.

## Initialize R environment
**The following applies when working on this course elsewhere and can be ignored when working on the JupyterHub:**

Unzip the directory you received from the course assistants and change the R working directory into the unzipped folder named 'burnhot' using `setwd("path/to/burnhot")`.

Alternatively, download the repository from github in a shell `git clone https://github.com/ikmb/burnhot.git` or from the website of the repo <https://github.com/ikmb/burnhot>.

Change the working directory in the R console into the burnhot directory `setwd("path/to/burnhot")`.

Before installing R libraries, you need to install cairolib on your system using bash ```apt-get install libcairo2-dev```.

**Today you can execute the cells within this Jupyter notebook directly. Select for this the 'rnaseq_microbiome'-kernel. Where all the libraries needed are preinstalled.**

Next, load necessary external libraries.

In [ ]:
library(data.table)
library(tidyverse)
library(ggrepel)
library(DESeq2)
library(ComplexHeatmap)
library(RColorBrewer)
library(topGO)
library(Rgraphviz)
library(genefilter)
library(geneplotter)
library(org.Hs.eg.db)
folder <- "~/bioinformatics_materials/bulkRNASeq/results"

if (!dir.exists(folder)){
  dir.create(folder)
}

## Inspect FastQC results

After running the pipeline, the tool *FastQC* outputs a report on the quality of the sequencing data. Using the tool *MultiQC*, the FastQC result and log output from other tools were compiled into one html document. 

#### Tasks

Open the report <multiqc_report1.html> in your file-browser and focus on the section "FastQC". It contains some quality checks, in which each sample is graded as "pass", "warn" or "fail". What do you find? Think for a minute if you can come up with an explanation. If you can't, no problem, as this will become clear in the following. 

## Read files and preprocessing

Before the tutorial, we have run the RNA-Seq pre-processing pipeline for you. The main result is a count table, with sample IDs (the patients) as columns, gene IDs as rows, and transcript counts as the values. From the clinics, we know who of those patients was diagnosed with COVID-19, and how severe their status was. This data is saved as tables in the project folder you downloaded.

We now load the data and bring it into a format that we can run the analysis functions on. 

In [ ]:
##Read sample info file
covid_info_file <- read.csv("~/bioinformatics_materials/bulkRNASeq/data/Sequenced_sample_metadata_020720 (1).csv")
covid_info_file$Pseudotime <- as.character(covid_info_file$Pseudotime)
##Read raw count file
count_data <- read.csv("~/bioinformatics_materials/bulkRNASeq/data/COVID_control_raw_counts_070720 (1).txt", sep = '\t')
count_data <- count_data[, as.character(covid_info_file$RNAseqID)]
##Format sample col data, the patient data
col_data <- covid_info_file
rownames(col_data) <- col_data$RNAseqID
col_data$Gender <- as.factor(col_data$Gender)
col_data$Disease.traj <- as.factor(col_data$Disease.traj)
col_data$Pseudotime <- as.factor(as.character(col_data$Pseudotime))

#### Tasks

1. Now, use the function ```head()``` to get an impression what your data actually look like. What do you need to type to get a first glance at the data?

We can have a brief look at the sample info table now and make sure that the samples in the count table have the same naming pattern as those in the sample info table.

In [ ]:
head(col_data)
head(count_data)
#View(col_data) #Run to display the whole phenotype table. 

In [ ]:
# Run your code here:
head(col_data)

2. Which function allows you a quick look at the end of a table? Run ```?head``` if you don't know the answer.

In [ ]:
tail(col_data)
tail(count_data)

In [ ]:
# Run your code here:


It is always a good idea to check if the numbers of columns and rows are as you expect.

In [ ]:
dim(col_data)
dim(count_data)

3. Are the number of rows in the sample info table same as the number of columns in the count table? What does the number of rows in the count table signify?

It is the number of genes that reads were assigned to during the feature counting step. It is the total number of genes and other features in our annotation.

## Exploratory data analysis with PCA and heatmaps

We would like to get an overview of our data, to see if samples from the same stage of the disease are more similar to each other than to healthy controls.

Principal Component Analysis (PCA) is a statistical method to reduce the dimensionality of a dataset. Here, we want to project our count data into a 2-dimensional space on our screen. The first principal component describes a vector in the real coordinate space which minimizes the average square distance of the data points to the line. Each subsequent component tries the same, but has to be orthogonal to all other vectors (<https://en.wikipedia.org/wiki/Principal_component_analysis>). Sometimes, principal components capture biological insights to our data, because they cluster samples with similar expression profiles.

#### Task

Before you run the section below, think about what you expect to see in the PCA plot for this dataset.

In [ ]:
##Calculate the principle components for the count table
pc <-prcomp(t(count_data))
pc_df <- as.data.frame(pc$x)
pc_df$Disease.traj <- factor(col_data$Disease.traj, 
                             levels = c("Healthy","Complicated (incremental)",
                                        "Critical","Complicated with hyperinflammatory syndrom",
                                        "Complicated (recovering)","Mild (recovering)",
                                        "recovery/asymptomatic","Recovered"), 
                             ordered = T)
pc_df$Patient <- col_data$PatientID

## check the fraction of the total variance that is captured by the principal components
summary(pc)

pseudotime_colors <- c("#A7A9AC","#E84F8C", "#7031B9", "#A04E9E", "#E65826", "#F99B1C", "#FDC077", "#51BBFE")
P <- ggplot(data = pc_df, mapping = aes(x=PC1, y=PC2, color=Disease.traj, label=Patient)) + 
  geom_point(size=5) + geom_text_repel(hjust=0, nudge_x = 2, nudge_y = 1, cex=3)
P <- P + scale_color_manual(values = pseudotime_colors)
P <- P + theme_bw() + theme(axis.text = element_text(size = 20), axis.title = element_text(size = 22))
##Set output size of the plot. 
options(repr.plot.width=10, repr.plot.height=8)
P
ggsave(P,file=paste0(folder,"/PCA.png"),height = 8,width = 10, units = "in", dpi = 300)

#### Tasks

Compare the PCA plot above with your expectation. Are the samples clustered by the disease status of the patient?

The clustering pattern of COVID-cases and controls is not as obvious as we might have expected. The controls are somewhat concentrated, but not convincingly. This might be because we have not transformed the count data yet. The DESeq2 library can help us out.

### Data transformation

The data transformation by DESeq2 consists of correcting for the variable number of sequenced molecules, for transcripts with too high variance that we consider a statistical artifact and putting the values on a log-scale.

In [ ]:
dds_counts <- DESeqDataSetFromMatrix(countData = count_data, colData = col_data, design = ~ Pseudotime)
dds_counts <- dds_counts[ rowSums(counts(dds_counts)) > 1, ]
dds_counts <- estimateSizeFactors(dds_counts)

vst_counts <- vst(dds_counts, blind = TRUE)
vst.norm.counts <- assay(vst_counts)

Let's try again with the PCA, using transformed data. 

In [ ]:
pc <-prcomp(t(vst.norm.counts))
pc_df <- as.data.frame(pc$x)
pc_df$Disease.traj <- factor(col_data$Disease.traj, levels = c("Healthy","Complicated (incremental)","Critical","Complicated with hyperinflammatory syndrom",
                                                               "Complicated (recovering)","Mild (recovering)","recovery/asymptomatic","Recovered"), ordered = T)
pc_df$Patient <- col_data$PatientID
pc_df$Gender <- col_data$Gender
pc_df$Age <- col_data$Age

summary(pc)
pseudotime_colors <- c("#A7A9AC","#E84F8C", "#7031B9", "#A04E9E", "#E65826", "#F99B1C", "#FDC077", "#51BBFE")
P <- ggplot(data = pc_df, mapping = aes(x=PC1, y=PC2, color=Disease.traj, label=Patient)) + 
  geom_point(size=5) + geom_text(hjust=0, nudge_x = 2, nudge_y = 1, cex=3) + xlab("PC1") + 
  ylab("PC2")
P <- P + scale_color_manual(values = pseudotime_colors)
P <- P + theme_bw() + theme(axis.text = element_text(size = 20), axis.title = element_text(size = 22))
P
ggsave(P,file=paste0(folder,"/PCA_vst.png"),height = 10,width = 10, units = "in", dpi = 300)

#### Tasks

1. Have a look at how the result changed. Does it support our hypothesis about differences between patients and controls?

2. Can you change the ggplot command above so that you can see the sex or the age of the patients from the color of the points? Hint: The ```scale_color_manual()``` might give funny results when used with Age as variable. 

In [ ]:
P <- ggplot(data = pc_df, mapping = aes(x = PC1, y = PC2, color = Gender, label = Patient)) + 
  geom_point(size = 5) + 
  geom_text(hjust = 0, nudge_x = 2, nudge_y = 1, cex = 3) + 
  xlab("PC1") + 
  ylab("PC2")
P
P <- ggplot(data = pc_df, mapping = aes(x = PC1, y = PC2, color = Age, label = Patient)) + 
  geom_point(size = 5) + 
  geom_text(hjust = 0, nudge_x = 2, nudge_y = 1, cex = 3) + 
  xlab("PC1") + 
  ylab("PC2")
P

In [ ]:
# Run your solution here:


## Differential expression analysis, using DESeq2

We have seen in the PCA that patients in a critical state of their COVID-19 disease cluster away from the controls. Now we want to know which transcripts are differentially expressed between healthy controls and patients with COVID-19 with different disease severity (pseudotime).

In [ ]:
#Read the file that maps gene Ensembl ID to gene name so that we can understand the genes better later on
ensg2gene <- read.table("~/bioinformatics_materials/bulkRNASeq/data/ensg2gene.txt", header = T, sep=",")
unique_ensg2gene <- subset(ensg2gene, duplicated(ensg2gene$ensembl_gene_id) == FALSE)
rownames(unique_ensg2gene) <- unique_ensg2gene$ensembl_gene_id

#We will recursively perform differential expression analysis between the controls and each of the pseudotime samples
pseudotime <- c("1", "2", "3", "4", "5", "6", "7")
output_folder <- folder

for (i in 1:length(pseudotime)) {
  
  #Filter info data for desired pseudotime
  col_data <- subset(covid_info_file, covid_info_file$Pseudotime %in% c('0', pseudotime[i]))
  rownames(col_data) <- col_data$RNAseqID
  col_data$Pseudotime <- as.factor(as.character(col_data$Pseudotime))
  
  #Filter count data for desired pseudotime
  selected_count_data <- count_data[, as.character(col_data$RNAseqID)]
  
  #Create deseq object
  dds_counts <- DESeqDataSetFromMatrix(countData = selected_count_data, colData = col_data, design = ~ Pseudotime)
  dds_counts <- dds_counts[ rowSums(counts(dds_counts)) > 1, ]
  dds_counts <- estimateSizeFactors(dds_counts)
  
  #Run DESeq
  dds <- DESeq(dds_counts, betaPrior = FALSE)
  res <- results(dds, independentFiltering = TRUE, alpha = 0.05)
  res_sorted <- res[order(res$padj), ]
  
  #Add gene symbols
  res_sorted$gene <- unique_ensg2gene[rownames(res_sorted), ]$hgnc_symbol
    
  fwrite(as.data.table(res_sorted, keep.rownames="ensembl_gene_id"), file = file.path(folder, paste("/DESeq2result_genenames_Control_vs_COVID_", pseudotime[i], ".txt", sep = '')), 
              sep="\t", quote=FALSE)  
  #write.table(res_sorted, file = file.path(folder, paste("/DESeq2result_genenames_Control_vs_COVID_", pseudotime[i], ".txt", sep = '')), 
  #            sep="\t", quote=FALSE)

}

You can load each table into the memory using the following commands:

In [ ]:
#change the 1 for a number from 1-7 to get the other results.
res_sorted<-fread(paste0(folder,"/DESeq2result_genenames_Control_vs_COVID_1.txt"))
res_sorted[1:20,]

#### Tasks

1. Try to find the gene with the strongest expression in the dataset (baseMean). The command ```res_sorted <- res[order(res$padj), ]``` has been used before to sort the table. Can you adapt the command so you get the strongest expressed genes? 

Hint: Type ```?base::order``` if you need information how to tweak the behavior of the function. Function ```head()``` is also helpful.

In [ ]:
head(res_sorted[order(res_sorted$baseMean,decreasing = T)])
# or
tail(res_sorted[order(res_sorted$baseMean)])

In [ ]:
# Run your solution here:

2. What is the transcript coding for? Look it up for example here: <https://www.genecards.org>. 

3. Look at the level of expression of the top two genes. Explain your FastQC results from the beginning considering this observation.

#### Advanced

1. Explain the gene's effect on the resolution of the experiment. What could you do to improve the resolution?

### Volcano plot

A volcano plot is a visualization of the significance and signed fold-change of genes in a differential expression experiment. It somehow resembles a volcano eruption with the most significant genes as rocks tossed into the sky.

In [ ]:
plot_data <- data.frame()

#Read each deseq out file and filter for significant differentially expressed genes
for (i in 1:length(pseudotime)) {
  input_folder <- folder
  result <- read.csv(file.path(folder, paste("DESeq2result_genenames_Control_vs_COVID_", pseudotime[i], ".txt", sep = '')), 
                     sep = '\t')
  result <- subset(result, result$padj < 0.95 & result$baseMean > 100 & abs(result$log2FoldChange) > 0.01)
  result$pseudotime <- pseudotime[i]
  result <- rownames_to_column(result, "Gene_ID")
  plot_data <- rbind(plot_data, result)
  
}

#Select interesting genes to label
gene_to_label <- c("MAOA", "HTRA3", "ERG", "TMEM119", "INHBB", "PTX3", "FCGR1CP", "PDZK1IP1", "FHL2", "NRIR", "SLC4A1", "C1QB", "TESC", "MERTK", 
                   "IDO1", "P2RY14", "F13A1", "STAT1", "GATA2", "NFKBIA", "TREML1", "NCR1", "NLRP6", "NLRP3", "JAK3", "TXK", "IL5RA", "SATB1-AS1", 
                   "SHARPIN", "TNFRSF10C", "CXCR1", "ATF6", "IFI16", "AHR", "IL13RA1", "C5AR1", "CSF2RA", "FYN", "IGHG1", "JCHAIN", "TOP2A", 
                   "IFI27", "BIRC5", "IGHA1", "IL1R2", "DEFA4", "DEFA3", "LTF", "RORC", "IFIT1B", "AIM2", "IL24", "CD38", "TNFRSF13C", "ERBB2", 
                   "TCF7", "CIITA", "TGFBI", "SIGIRR", "IGHA2", "CDC6", "CHI3L1", "HLA-DPA1", "HLA-DMB", "CD4", "IL6ST", "CD274", "C3AR1", "ADAM17", 
                   "C1QA", "CARD11", "CD14", "BCL3", "TICAM1")
gene_to_label2 <- c("HLA-DMA", "HLA-DMB", "HLA-DOA", "HLA-DPB1", "HLA-DPB", "HLA-DQA1", "HLA-DQB1", "HLA-DR1", 
                    "HLA-DRB1", "HLA-DRB5")

plot_data$color <- ifelse(plot_data$log2FoldChange > 1 & plot_data$padj < 0.05, "red", 
                             ifelse(plot_data$log2FoldChange < -1 & plot_data$padj < 0.05, "blue", 
                                    ifelse(plot_data$padj < 0.05, "black", "grey")))

volcano1 <- ggplot(data = plot_data[plot_data$pseudotime=="1",], mapping=aes(x= log2FoldChange, y=-log10(padj), color=color)) + 
  geom_point() +
  geom_label_repel(data = subset(plot_data, plot_data$gene %in% gene_to_label & abs(plot_data$log2FoldChange) > 2 & pseudotime=="1"), 
                   aes(x=log2FoldChange, y=-log10(padj), label=gene), 
                   color="red", nudge_x = -0.5, min.segment.length = 0) + 
  geom_label_repel(data = subset(plot_data, plot_data$gene %in% gene_to_label2& abs(plot_data$log2FoldChange) > 1 & pseudotime == "1"), 
                   aes(x=log2FoldChange, y= -log10(padj), label=gene), 
                   color="red", nudge_x = -0.5, min.segment.length = 0)
volcano1 <- volcano1 + geom_vline(xintercept = 0, lty=3) + geom_vline(xintercept = 1, lty=4) + geom_vline(xintercept = -1, lty=4) + 
  geom_hline(yintercept = 1.30103, lty=4) + geom_hline(yintercept = 3, lty=4)
volcano1 <- volcano1 + scale_color_manual(values = c('#606060','#004C99','#C0C0C0','#990000'))
volcano1 <- volcano1 + theme_bw() + theme(axis.text = element_text(size = 20), axis.title = element_text(size = 22))
volcano1
ggsave(filename=paste0(folder,"/volcano.png"),plot=volcano1)

#### Tasks

1. Change the code so you see the volcano plot of pseudotime 2 compared to controls.

In [ ]:
#change the filter in the following line, which selects the data: plot_data[plot_data$pseudotime=="2",]
#this selects the data at pseudo-timepoint 2
#volcano1 <- ggplot(data = plot_data[plot_data$pseudotime=="2",], mapping=aes(x= log2FoldChange, y=-log10(padj), color=color))

In [ ]:
#Insert your code here:

In [ ]:
# change the pseudotime=="1" to pseudotime=="2" et voilà.  
volcano1 <- ggplot(data = plot_data[plot_data$pseudotime=="2",], mapping=aes(x= log2FoldChange, y=-log10(padj), color=color)) +
  geom_point() +
  geom_label_repel(data = subset(plot_data, plot_data$gene %in% gene_to_label & abs(plot_data$log2FoldChange) > 2 & pseudotime=="2"), 
                   aes(x=log2FoldChange, y=-log10(padj), label=gene), 
                   color="red", nudge_x = -0.5, min.segment.length = 0) + 
  geom_label_repel(data = subset(plot_data, plot_data$gene %in% gene_to_label2& abs(plot_data$log2FoldChange) > 1 & pseudotime == "2"), 
                   aes(x=log2FoldChange, y= -log10(padj), label=gene), 
                   color="red", nudge_x = -0.5, min.segment.length = 0)
volcano1 <- volcano1 + geom_vline(xintercept = 0, lty=3) + geom_vline(xintercept = 1, lty=4) + geom_vline(xintercept = -1, lty=4) + 
  geom_hline(yintercept = 1.30103, lty=4) + geom_hline(yintercept = 3, lty=4)
volcano1 <- volcano1 + scale_color_manual(values = c('#606060','#004C99','#C0C0C0','#990000'))
volcano1 <- volcano1 + theme_bw() + theme(axis.text = element_text(size = 20), axis.title = element_text(size = 22))
volcano1

2. Change the code so you only see genes which have a negative log2foldchange.

In [ ]:
# in the line below, add a filtering condition log2FoldChange < 0 to the data and connect the two filters with the & -sign.
#volcano1 <- ggplot(data = plot_data[plot_data$pseudotime=="1" & log2FoldChange < 0,], mapping=aes(x= log2FoldChange, y=-log10(padj), color=color))

In [ ]:
#Insert your code here:

In [ ]:
# abs() gives the absolute value of the input. We remove the function and set the comparison to "plot_data$log2FoldChange < 0"
# For the labeling of significant genes, we can filter by removing abs() and changing to "plot_data$log2FoldChange < -2"
volcano1 <- ggplot(data = plot_data[plot_data$pseudotime=="1" & plot_data$log2FoldChange < 0,], mapping=aes(x= log2FoldChange, y=-log10(padj), color=color)) +
  geom_point() +
  geom_label_repel(data = subset(plot_data, plot_data$gene %in% gene_to_label & plot_data$log2FoldChange < -2 & pseudotime=="1"), 
                   aes(x=log2FoldChange, y=-log10(padj), label=gene), 
                   color="red", nudge_x = -0.5, min.segment.length = 0) + 
  geom_label_repel(data = subset(plot_data, plot_data$gene %in% gene_to_label2 & plot_data$log2FoldChange < -1 & pseudotime == "1"), 
                   aes(x=log2FoldChange, y= -log10(padj), label=gene), 
                   color="red", nudge_x = -0.5, min.segment.length = 0)
volcano1 <- volcano1 + geom_vline(xintercept = 0, lty=3) + geom_vline(xintercept = 1, lty=4) + geom_vline(xintercept = -1, lty=4) + 
  geom_hline(yintercept = 1.30103, lty=4) + geom_hline(yintercept = 3, lty=4)
volcano1 <- volcano1 + scale_color_manual(values = c('#606060','#004C99','#C0C0C0','#990000'))
volcano1 <- volcano1 + theme_bw() + theme(axis.text = element_text(size = 20), axis.title = element_text(size = 22))
volcano1

## Gene expression heatmaps

Heatmaps are an effective way to visualize gene expression of multiple genes across different samples or groups. The ComplexHeatmap package <https://jokergoo.github.io/ComplexHeatmap-reference/book/> provides multiple ways to add annotations to the heatmap as well as clustering options. Normalized genes expression data should be used to create Heatmaps. Here we plot a heatmap for the 100 most differentially expressed genes grouped by different disease severities (pseudotimes). 

In [ ]:
top100 <- res_sorted[1:100,]
top100 <- subset(top100, top100$gene != '')

col_data <- covid_info_file
rownames(col_data) <- col_data$RNAseqID
col_data$Gender <- as.factor(col_data$Gender)
col_data$Disease.traj <- as.factor(col_data$Disease.traj)
col_data$Pseudotime <- as.factor(as.character(col_data$Pseudotime))
col_data$Sample_name <- paste(col_data$Name, col_data$Pseudotime, sep = '_')

#Normalize the expression counts
dds_counts <- DESeqDataSetFromMatrix(countData = count_data, colData = col_data, design = ~ Pseudotime)
dds_counts <- dds_counts[ rowSums(counts(dds_counts)) > 1, ]
dds_counts <- estimateSizeFactors(dds_counts)
normalized_counts <- counts(dds_counts, normalized = TRUE)

top100_expression_data <- normalized_counts[as.numeric(rownames(top100)), ]
colnames(top100_expression_data) <- col_data$Sample_name

#Calculate mean expression level for each pseudotime
vecPT <- str_split_fixed(colnames(top100_expression_data), '_', 4)[, 4]
vecUniquePT <- unique(vecPT)

matDataHeat <- do.call(cbind, lapply(vecUniquePT, function(pt) {
  vecidxCols <- which(vecPT %in% pt)
  if (length(vecidxCols) > 1) {
    return(rowMeans(top100_expression_data[, vecidxCols], na.rm = TRUE))
  } else {
    return(top100_expression_data[, vecidxCols])
  }
}))
colnames(matDataHeat) <- vecUniquePT
rownames(matDataHeat) <- top100$gene

matDataHeat <- matDataHeat[, c("0", "1", "2", "3", "4", "5", "6", "7")]

#Scale the data row-wise to highlight the genes expression differences across pseudotimes
matDataHeat <- t(scale(t(matDataHeat)))
names(pseudotime_colors) <- colnames(matDataHeat)

#Annotate the columns by pseudotimes and plot heatmap
col_ha = HeatmapAnnotation(df = data.frame(Pseudotime = colnames(matDataHeat)),
                           col = list(Pseudotime = pseudotime_colors), 
                           gp = gpar(col = "black"))
Heatmap(matDataHeat, col=rev(brewer.pal(11, "RdYlBu")), top_annotation = col_ha, cluster_columns = FALSE, row_names_gp = gpar(fontsize = 8), border = TRUE)

#### Tasks

In the code for the above graph, you find this line ```matDataHeat <- t(scale(t(matDataHeat)))```. It performs a transformation on the averaged count data to be plotted. 
1. Find out what it does, try out and explain why we need it for an informative figure output.

In [ ]:
#You can use the ?-sign to find out what a function does.
?scale

The data is scaled on gene level. This means that the mean value is subtracted and the difference is divided by the standard deviation of the expression values. By commenting out the line, you see that the expression level then dominates the clustering of the genes, not similarity in expression pattern. Scaling is a useful transformation if features to be compared have widely different means.

#### Expert task

1. Plot a heatmap with all individual samples grouped by different pseudotimes.

In [ ]:
#order the sample data by pseudotime
col_data_copy <- copy(col_data)
setorder(col_data_copy, Pseudotime)
#reorder the columns of the expression data accordingly
top100_expression_data_sorted <- top100_expression_data[,col_data_copy$Sample_name]
#give gene names as rownames, overwriting the ENSG... naming
rownames(top100_expression_data_sorted) <- top100$gene
#scale the expression data on the gene level, to make them comparable across samples
top100_expression_data_sorted <- t(scale(t(top100_expression_data_sorted)))
#prepare the annotation
name_colors <- col_data_copy$color_code
names(name_colors) <- col_data_copy$Sample_name
names(pseudotime_colors) <- colnames(matDataHeat)
col_ha2 <- HeatmapAnnotation(df = data.frame(#Name = col_data$Sample_name,
                           Pseudotime = col_data_copy$Pseudotime),
                           col = list(Pseudotime = pseudotime_colors), 
                           gp = gpar(col = "black"))
Heatmap(top100_expression_data_sorted, col=rev(brewer.pal(11, "RdYlBu")), top_annotation = col_ha2, cluster_columns = FALSE, row_names_gp = gpar(fontsize = 8), border = TRUE)

## Enrichment analysis of differentially expressed genes

In [ ]:
res_sorted <- read.table(paste0(folder,"/DESeq2result_genenames_Control_vs_COVID_1.txt"), header = TRUE, sep = '\t')
write(res_sorted$gene[1:100],file = paste0(folder, "/top100genes.txt"))

You now have a list of significant differentially expressed genes in patients with complicated incremental disease. You can use <https://maayanlab.cloud/Enrichr/> to check if you can find an enriched term that could give you insight on the underlying processes. Tip: Check the "Pathways" and "Ontologies" tabs. Click on the red captions of the database name. You get to a view where you can hover over the bars to check the q-value of the enrichment. The q-value is a multiple testing corrected p-value and should be less than 0.05 to speak of a significant enrichment.

#### Tasks

Do you find a term which seems related to COVID? Which one?

### TopGO

As more accurate approach, you can use a bioconductor package called topGO to perform the GO Term enrichment analysis. topGO package provides tools for testing GO terms while accounting for the topology of the GO graph. It can also account for the gene background (genes that were detected in the experiment at least once). Different test statistics and different methods for eliminating local similarities and dependencies between GO terms can be implemented and applied.

In [ ]:
rownames(res_sorted) <- res_sorted$ensembl_gene_id
#filter out genes that were never detected
res_sorted <- subset(res_sorted, res_sorted$baseMean > 0)
degs <- subset(res_sorted, res_sorted$padj < 0.05 & abs(res_sorted$log2FoldChange) > 1)
top100 <- rownames(degs[1:100,])
all_genes <- rownames(res_sorted)
#top100 <- degs$ensembl_gene_id[1:100]
#all_genes <- res_sorted$ensembl_gene_id

inUniverse <- all_genes %in% all_genes
inSelection <- all_genes %in% top100 
alg <- factor( as.integer( inSelection[inUniverse] ) )
names(alg) <- all_genes[inUniverse]

## prepare data
tgd <- new( "topGOdata", ontology="BP", allGenes = alg, nodeSize=5,
            annot=annFUN.org, mapping="org.Hs.eg.db", ID = "ensembl" )

## run tests
resultTopGO.elim <- runTest(tgd, algorithm = "elim", statistic = "Fisher" )
resultTopGO.classic <- runTest(tgd, algorithm = "classic", statistic = "Fisher" )

## look at results
if(length(nodes(graph(tgd))) < 200){
  tab <- GenTable( tgd, Fisher.elim = resultTopGO.elim, 
                   Fisher.classic = resultTopGO.classic,
                   orderBy = "Fisher.elim" , topNodes = length(nodes(graph(tgd))))
}else{
  tab <- GenTable( tgd, Fisher.elim = resultTopGO.elim, 
                   Fisher.classic = resultTopGO.classic,
                   orderBy = "Fisher.elim" , topNodes = 200)}

printGraph(tgd, resultTopGO.elim, firstSigNodes = 5, fn.prefix = paste0(folder,"/"), useInfo = "all", pdfSW = TRUE)
printGraph(tgd, resultTopGO.classic, firstSigNodes = 15, fn.prefix = paste0(folder,"/"), useInfo = "all", pdfSW = TRUE)
tab

#### Tasks

Do you see any biological processes that are related to the immune response or inflammation?
1. Change the code to identify the GO terms that describe molecular function (MF) and/or cellular components (CC) instead of biological processes (BP). Hint: change 'ontology' while creating the topGOdata object.
2. Change the code to identify GO terms that are enriched in genes that are downregulated in COVID-19.

#### Expert questions

1. Can you see any difference in the p-values generated by the Fisher classic algorithm and Fisher elim algorithm? Check the graphs for the difference in p-values at the different levels of the GO graph.
2. Is it correct to calculate the enrichment of GO terms for our genes of interest against all expressed genes? Might it be better if we compare genes that have similar expression?

Run the code and find out!

The function genefinder from the genefilter package will be used to find background genes that are similar in expression to our genes of interest. The function tries to identify 10 genes for each gene of interest that match its expression.

In [ ]:
overallBaseMean <- as.matrix(res_sorted[, c("baseMean"), drop = F])

sig_idx <- match(top100, rownames(overallBaseMean))

backG <- c()

for(i in sig_idx){
  ind <- genefinder(overallBaseMean, i, 10, method = "manhattan")[[1]]$indices
  backG <- c(backG, ind)
  
}

We then check whether the background has roughly the same distribution of average expression strength as the foreground by plotting the densities.

In [ ]:
backG <- unique(backG)
backG <- rownames(overallBaseMean)[backG]
backG <- setdiff(backG, top100)

multidensity( list( 
  all= log2(res_sorted[,"baseMean"]) ,
  foreground =log2(res_sorted[top100, "baseMean"]), 
  background =log2(res_sorted[backG, "baseMean"])), 
  xlab="log2 mean normalized counts", main = "Matching for enrichment analysis")

Run topGO with unbiased background.

In [ ]:
inUniverse <- all_genes %in% c(top100, backG)
inSelection <- all_genes %in% top100 
alg <- factor( as.integer( inSelection[inUniverse] ) )
names(alg) <- all_genes[inUniverse]

## prepare data
tgd <- new( "topGOdata", ontology="BP", allGenes = alg, nodeSize=5,
            annot=annFUN.org, mapping="org.Hs.eg.db", ID = "ensembl" )

## run tests
resultTopGO.elim <- runTest(tgd, algorithm = "elim", statistic = "Fisher" )
resultTopGO.classic <- runTest(tgd, algorithm = "classic", statistic = "Fisher" )

## look at results
if(length(nodes(graph(tgd))) < 200){
  tab <- GenTable( tgd, Fisher.elim = resultTopGO.elim, 
                   Fisher.classic = resultTopGO.classic,
                   orderBy = "Fisher.elim" , topNodes = length(nodes(graph(tgd))))
}else{
  tab <- GenTable( tgd, Fisher.elim = resultTopGO.elim, 
                   Fisher.classic = resultTopGO.classic,
                   orderBy = "Fisher.elim" , topNodes = 200)}

printGraph(tgd, resultTopGO.elim, firstSigNodes = 5, fn.prefix = paste0(folder,"/selected_background"), useInfo = "all", pdfSW = TRUE)
printGraph(tgd, resultTopGO.classic, firstSigNodes = 15, fn.prefix = paste0(folder,"/selected_background"), useInfo = "all", pdfSW = TRUE)

We next find the genes associated with the enriched GO terms.

In [ ]:
for(i in 1:nrow(tab)){
  go_id <- as.vector(tab[i,1])
  genes_in_cat <- intersect(unlist(genesInTerm(tgd, go_id)), top100)
  gene_sym_in_cat <- unique_ensg2gene[as.character(genes_in_cat), "hgnc_symbol"]
  gene_sym_in_cat_str <- ""
  
  if(length(genes_in_cat) > 0){
    for(j in 1:length(gene_sym_in_cat)){
      gene_sym_in_cat_str <- paste(gene_sym_in_cat_str, gene_sym_in_cat[j], sep = ',')
    }
  }
  if (gene_sym_in_cat_str == ""){
    gene_sym_in_cat_str <- NA
  }

  tab$Genes[i] <- gene_sym_in_cat_str

}
tab

## Misc

This code was used to download the ensemblID to gene name matching table.

In [ ]:
# library('biomaRt')
# ensembl = useEnsembl(biomart = "genes")
# ensembl = useDataset("hsapiens_gene_ensembl",mart=ensembl)
# #ensembl = useMart("ensembl", dataset = "hsapiens_gene_ensembl")
# genes <- rownames(count_data)
# #df<-df[,-4]
# G_list <- getBM(filters= "ensembl_gene_id", attributes= c("ensembl_gene_id","hgnc_symbol"),values=genes,mart= ensembl)
# G_DT<-as.data.table(G_list)
# data.table::fwrite(G_DT, col.names = T,file = "ensg2gene.txt")
# merge(df,G_list,by.x="gene",by.y="ensembl_peptide_id")